In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Add parent directory to sys.path in order to find common module
import sys
sys.path.insert(0,'../')

import common.datautils as datautils
import common.featuresutils as featuresutils

In [ ]:
flag_effect_data = datautils.load_csv_results(
    'flag_effects.csv',
    ['Benchmark','DisabledFlag', 'Flags', 'Success']
)
flag_effect_data = flag_effect_data[flag_effect_data["DisabledFlag"] != '-O3']

flag_effect_data.head()

In [ ]:
benchmarks = flag_effect_data["Benchmark"].unique()
print(benchmarks)

In [ ]:
ce_data = datautils.load_ce_results('CE.results.zip')
best_known = []

for benchmark in benchmarks:
    best_known.append(datautils.best_configuration_data('Energy', benchmark, ce_data, ['Benchmark', 'Flags', 'Energy', 'Time']))
    
best_known = pd.DataFrame(best_known)
best_known.set_index('Benchmark', drop=True, inplace=True)    

In [ ]:
improvement_factor = 0.2


In [ ]:
def flag_relative_data():
    relative_flag_effects = flag_effect_data.copy()

    for benchmark in benchmarks:
        benchmark_data = relative_flag_effects[relative_flag_effects["Benchmark"] == benchmark]
        
        best_known_energy = best_known.loc[benchmark]['Energy']
        best_known_time = best_known.loc[benchmark]['Time']

        relative_flag_effects.loc[relative_flag_effects["Benchmark"] == benchmark, "Energy"] = best_known_energy / benchmark_data["Energy"]
        relative_flag_effects.loc[relative_flag_effects["Benchmark"] == benchmark, "Time"] = best_known_time / benchmark_data["Time"]

        
    relative_flag_effects.loc[:, 'Energy'] = (1 - np.round(relative_flag_effects["Energy"], 2)) * 100
    relative_flag_effects.loc[:, 'Time'] = (1 - np.round(relative_flag_effects["Time"], 2)) * 100
    
    return relative_flag_effects

relative_data = flag_relative_data()

In [ ]:
average_data = relative_data.groupby(['DisabledFlag', 'Success'], as_index=False).agg({'Energy':'mean', 'Time':'mean'})
average_data[(average_data["Energy"] > average_data["Time"]) & (average_data["Time"] > 0)].sort_values(['Energy', 'Time'], ascending=False).head(20)